In [1]:
import numpy as np

docs1 = ['I love you','Thank you very much','how are you','Im happy to see you','Goodbye, see you later']
docs2 =['Main tumse pyaar karta hoon','Bahut dhanyavaad','Aap kaise hain','Main aapko dekhkar khush hoon','Alvida, phir milenge']

In [26]:
# Add <START> and <END> tokens to target sequences
target_texts = ["<START> " + text + " <END>" for text in docs2]

In [27]:
target_texts

['<START> Main tumse pyaar karta hoon <END>',
 '<START> Bahut dhanyavaad <END>',
 '<START> Aap kaise hain <END>',
 '<START> Main aapko dekhkar khush hoon <END>',
 '<START> Alvida, phir milenge <END>']

In [28]:
input_texts  = docs1

In [29]:
input_texts

['I love you',
 'Thank you very much',
 'how are you',
 'Im happy to see you',
 'Goodbye, see you later']

In [30]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [31]:
# Tokenizer for English (input)
input_tokenizer = Tokenizer(num_words=15, filters='')
input_tokenizer.fit_on_texts(input_texts)
input_sequences = input_tokenizer.texts_to_sequences(input_texts)
input_sequences = pad_sequences(input_sequences, maxlen=10, padding='post')

In [32]:
input_sequences

array([[ 3,  4,  1,  0,  0,  0,  0,  0,  0,  0],
       [ 5,  1,  6,  7,  0,  0,  0,  0,  0,  0],
       [ 8,  9,  1,  0,  0,  0,  0,  0,  0,  0],
       [10, 11, 12,  2,  1,  0,  0,  0,  0,  0],
       [13,  2,  1, 14,  0,  0,  0,  0,  0,  0]], dtype=int32)

In [33]:
# Tokenizer for Spanish (output)
target_tokenizer = Tokenizer(num_words=19, filters='')
target_tokenizer.fit_on_texts(target_texts)
target_sequences = target_tokenizer.texts_to_sequences(target_texts)
target_sequences = pad_sequences(target_sequences, maxlen=10, padding='post')

In [34]:
target_sequences

array([[ 1,  3,  5,  6,  7,  4,  2,  0,  0,  0],
       [ 1,  8,  9,  2,  0,  0,  0,  0,  0,  0],
       [ 1, 10, 11, 12,  2,  0,  0,  0,  0,  0],
       [ 1,  3, 13, 14, 15,  4,  2,  0,  0,  0],
       [ 1, 16, 17, 18,  2,  0,  0,  0,  0,  0]], dtype=int32)

In [35]:
# Decoder Input (shifted target sentences for teacher forcing)
decoder_input_data = np.array(target_sequences[:, :-1])  # Remove last token
decoder_target_data = np.array(target_sequences[:, 1:])  # Remove first token

In [36]:
decoder_input_data

array([[ 1,  3,  5,  6,  7,  4,  2,  0,  0],
       [ 1,  8,  9,  2,  0,  0,  0,  0,  0],
       [ 1, 10, 11, 12,  2,  0,  0,  0,  0],
       [ 1,  3, 13, 14, 15,  4,  2,  0,  0],
       [ 1, 16, 17, 18,  2,  0,  0,  0,  0]], dtype=int32)

In [37]:
decoder_target_data

array([[ 3,  5,  6,  7,  4,  2,  0,  0,  0],
       [ 8,  9,  2,  0,  0,  0,  0,  0,  0],
       [10, 11, 12,  2,  0,  0,  0,  0,  0],
       [ 3, 13, 14, 15,  4,  2,  0,  0,  0],
       [16, 17, 18,  2,  0,  0,  0,  0,  0]], dtype=int32)

In [38]:
# Vocabulary sizes
input_vocab_size = len(input_tokenizer.word_index) + 1
target_vocab_size = len(target_tokenizer.word_index) + 1

In [39]:
print("Encoder Input Shape:", input_sequences.shape)
print("Decoder Input Shape:", decoder_input_data.shape)
print("Decoder Target Shape:", decoder_target_data.shape)

Encoder Input Shape: (5, 10)
Decoder Input Shape: (5, 9)
Decoder Target Shape: (5, 9)


In [42]:
max_len=10

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding

latent_dim = 10  # LSTM cell size, number of time step

# Encoder
encoder_inputs = Input(shape=(max_len,))
enc_emb = Embedding(input_vocab_size, latent_dim, mask_zero=True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(max_len - 1,))
dec_emb = Embedding(target_vocab_size, latent_dim, mask_zero=True)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(target_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the full Seq2Seq model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Summary
model.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_7       │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_8       │ (None, 9)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_8         │ (None, 10, 10)    │        150 │ input_layer_7[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_6         │ (None, 10)        │          0 │ input_layer_7[0]… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_9         │ (None, 9, 10)     │        190 │ input_layer_8[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_9 (LSTM)       │ [(None, 10),      │        840 │ embedding_8[0][0… │
│                     │ (None, 10),       │            │ not_equal_6[0][0] │
│                     │ (None, 10)]       │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_10 (LSTM)      │ [(None, 9, 10),   │        840 │ embedding_9[0][0… │
│                     │ (None, 10),       │            │ lstm_9[0][1],     │
│                     │ (None, 10)]       │            │ lstm_9[0][2]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 9, 19)     │        209 │ lstm_10[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,229 (8.71 KB)

 Trainable params: 2,229 (8.71 KB)

 Non-trainable params: 0 (0.00 B)

In [58]:
# Training the model
model.fit(
    [input_sequences, decoder_input_data],  
    decoder_target_data,
    batch_size=32,
    epochs=100,
    validation_split=0.2
)


Epoch 1/100


d:\a27_YEARS_OLD\RNN_GRU_LSTM_TRANSFORMER\venv\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_64', 'keras_tensor_70']. Received: the structure of inputs=('*', '*')
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.0833 - loss: 2.9441 - val_accuracy: 0.1111 - val_loss: 2.9449
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.1944 - loss: 2.9425 - val_accuracy: 0.3333 - val_loss: 2.9440
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.2778 - loss: 2.9409 - val_accuracy: 0.3333 - val_loss: 2.9431
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.3611 - loss: 2.9393 - val_accuracy: 0.2222 - val_loss: 2.9422
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.3611 - loss: 2.9377 - val_accuracy: 0.2222 - val_loss: 2.9413
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.3056 - loss: 2.9361 - val_accuracy: 0.2222 - val_loss: 2.9404
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.3056 - loss: 2.9344 - val_accuracy: 0.0000e+00 - val_loss: 2.9395
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.3056 - loss: 2.9328 - val_accuracy: 0.0000e+00 - val_loss: 2.938

In [70]:
target_seq = np.zeros((1, max_len - 1))

In [71]:
target_seq

array([[0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [72]:
target_seq[0,0]

np.float64(0.0)

In [ ]:
def translate_sentence(input_sentence):
    # Convert input sentence to sequence
    input_seq = input_tokenizer.texts_to_sequences([input_sentence])
    input_seq = pad_sequences(input_seq, maxlen=max_len, padding='post')
    input_seq = np.array(input_seq).reshape(1,10,1)
    print(input_seq.shape)
    print(input_seq)
    
    
    inference_enc_emb = Embedding(input_vocab_size, latent_dim, mask_zero=True)(tf.convert_to_tensor([[ 3,  4,  1,  0,  0,  0,  0,  0,  0,  0]]))

    # Get encoder states
    inference_outputs, inference_state_h, inference_state_c = encoder_lstm(inference_enc_emb)
    inf_encoder_states = [inference_state_h, inference_state_c]
    print(inf_encoder_states)

    # Start with "<START>" token
    target_seq = np.zeros((1, max_len - 1))
    target_seq[0, 0] = target_tokenizer.word_index['<start>']

    decoded_sentence = []
    
    for _ in range(max_len - 1):
        output_tokens, h, c = decoder_lstm(dec_emb, initial_state=inf_encoder_states)
        
        
        print(tf.constant(output_tokens))
        
        output_word_index = decoder_dense(output_tokens)#[0, -1, :]
        # print(K.eval(output_word_index))
        # print(next(iter(output_word_index)))
        # output_word_index = np.argmax(K.eval(output_word_index)[0, -1, :])

        # Convert index to word
        word = target_tokenizer.index_word.get(output_word_index, "")
        if word == "<end>":
            break

        decoded_sentence.append(word)
        target_seq[0, _ + 1] = output_word_index
        enc_states = [h, c]  # Update states

    return " ".join(decoded_sentence)

# Example translation
print(translate_sentence("how are you"))  # Should return "hola"


(1, 10, 1)
[[[3]
  [4]
  [1]
  [0]
  [0]
  [0]
  [0]
  [0]
  [0]
  [0]]]
[<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[-0.11265229,  0.11997791,  0.10080139,  0.10074095,  0.02454397,
         0.10033459,  0.06446382,  0.00237495,  0.11137199,  0.13459839]],
      dtype=float32)>, <tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[-0.20331392,  0.22030897,  0.17768933,  0.18319166,  0.04820284,
         0.1867247 ,  0.11935928,  0.0045626 ,  0.22277631,  0.2557798 ]],
      dtype=float32)>]


ValueError: A KerasTensor is symbolic: it's a placeholder for a shape an a dtype. It doesn't have any actual numerical value. You cannot convert it to a NumPy array.